# Project 4 : Twitter Sentiment Analysis


To collect text data using Twitter API.

## Import Libraries

In [29]:
import numpy as np
import pandas as pd
import tweepy
import openpyxl
import json
from tweepy import OAuthHandler
from textblob import TextBlob
import re

## Define the Twitter API Keys

consumer_key

consumer_secret

access_token

access_token_secret

**The above keys are confidential, So masking it for submission**

In [4]:
consumer_key = "xxxx"
consumer_secret = "xxxx"
access_token = "xxxxx"
access_token_secret = "xxxx"

## Connect to twitter using API

In [9]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

## Test API in twitter to search for latest trending tweets : Raw format

In [16]:
query = "covid cases in chennai"

In [17]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x000001D507EF6848>, _json={'created_at': 'Sun Nov 22 13:56:54 +0000 2020', 'id': 1330510570295074818, 'id_str': '1330510570295074818', 'full_text': '@EPSTamilNadu @Vijayabaskarofl @narendramodi @MoHFW_INDIA @AmitShah this is how the cases go unnoticed not only in Tamilnadu but in other states also. Where govt  is not worried about health of people and has not developed infrastructure in healthcare https://t.co/m8Vvq37KOx', 'truncated': False, 'display_text_range': [0, 275], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'EPSTamilNadu', 'name': 'Edappadi K Palaniswami', 'id': 973935385557413889, 'id_str': '973935385557413889', 'indices': [0, 13]}, {'screen_name': 'Vijayabaskarofl', 'name': 'Dr C Vijayabaskar', 'id': 1019887446417215489, 'id_str': '1019887446417215489', 'indices': [14, 30]}, {'screen_name': 'narendramodi', 'name': 'Narendra Modi', 'id': 18839785, 'id_str': '18839785', 'indices': [31, 44]}, {'screen_nam

## Get the tweets and some Attributes into dataframe

In [18]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [19]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [20]:
stream(data=["covid cases in chennai"] , file_name='my_tweets')

In [24]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @smitha_tk: #TamilNadu district wise breaku...,Nikhil JP,299,5,Mumbai,False,0,3,2020-11-22 14:06:08
1,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Nikhil JP,299,5,Mumbai,False,0,2,2020-11-22 14:06:05
2,@EPSTamilNadu @Vijayabaskarofl @narendramodi @...,vasankj1,846,6,"India, Chennai",False,0,0,2020-11-22 13:56:54
3,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,2,2020-11-22 13:48:31
4,RT @smitha_tk: #TamilNadu district wise breaku...,Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,3,2020-11-22 13:48:28


In [25]:
df.shape

(347, 9)

## textblob functions to get Sentiment
Sentiment(polarity, subjectivity)

Polarity score: [-1 , +1]

Subjectivity: [0.0 , 1.0] 
where 0.0 is very Objective and 1.0 is very Subjective.

In [26]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [27]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [30]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [31]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @smitha_tk: #TamilNadu district wise breaku...,Nikhil JP,299,5,Mumbai,False,0,3,2020-11-22 14:06:08,RT tk TamilNadu district wise breakup of COVID...
1,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Nikhil JP,299,5,Mumbai,False,0,2,2020-11-22 14:06:05,RT tk 1 655 covid19 cases reported in TamilNad...
2,@EPSTamilNadu @Vijayabaskarofl @narendramodi @...,vasankj1,846,6,"India, Chennai",False,0,0,2020-11-22 13:56:54,INDIA this is how the cases go unnoticed not o...
3,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,2,2020-11-22 13:48:31,RT tk 1 655 covid19 cases reported in TamilNad...
4,RT @smitha_tk: #TamilNadu district wise breaku...,Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,3,2020-11-22 13:48:28,RT tk TamilNadu district wise breakup of COVID...


In [32]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @smitha_tk: #TamilNadu district wise breaku...,Nikhil JP,299,5,Mumbai,False,0,3,2020-11-22 14:06:08,RT tk TamilNadu district wise breakup of COVID...,Positive
1,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Nikhil JP,299,5,Mumbai,False,0,2,2020-11-22 14:06:05,RT tk 1 655 covid19 cases reported in TamilNad...,Neutral
2,@EPSTamilNadu @Vijayabaskarofl @narendramodi @...,vasankj1,846,6,"India, Chennai",False,0,0,2020-11-22 13:56:54,INDIA this is how the cases go unnoticed not o...,Negative
3,"RT @smitha_tk: 1,655 #covid19 cases reported i...",Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,2,2020-11-22 13:48:31,RT tk 1 655 covid19 cases reported in TamilNad...,Neutral
4,RT @smitha_tk: #TamilNadu district wise breaku...,Ashok M (மூ.அசோக்),17545,298,"chennai, India",False,0,3,2020-11-22 13:48:28,RT tk TamilNadu district wise breakup of COVID...,Positive


## Check tweets for Positive, Neutral and Negative Sentiments

In [39]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @smitha_tk: 1,655 #covid19 cases reported in #TamilNadu today

Of which 489 cases from #Chennai 

70,139 persons tested, 2,010 patients…

Clean tweet:
 RT tk 1 655 covid19 cases reported in TamilNadu today Of which 489 cases from Chennai 70 139 persons tested 2 010 patients

Sentiment of the tweet:
 Neutral


In [40]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @TOIChennai: Tamil Nadu: Passengers from cities with high Covid-19 cases go unmonitored https://t.co/AtVvtFt2xm

Clean tweet:
 RT Tamil Nadu Passengers from cities with high Covid 19 cases go unmonitored

Sentiment of the tweet:
 Positive


In [45]:
n = 2
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @EPSTamilNadu @Vijayabaskarofl @narendramodi @MoHFW_INDIA @AmitShah this is how the cases go unnoticed not only in… https://t.co/S1emdJY2jQ

Clean tweet:
 INDIA this is how the cases go unnoticed not only in

Sentiment of the tweet:
 Negative


In [46]:
df[df.Sentiment == 'Positive'].shape[0]

216

In [47]:
df[df.Sentiment == 'Neutral'].shape[0]

89

In [48]:
df[df.Sentiment == 'Negative'].shape[0]

42

In [ ]:
# End of Assignment